# Training Word2Vec for Arabic BERT Corpus

The purpouse of this notebook is to show how we can train a Word2Vec model for [Arabic Corpus dataset](https://www.kaggle.com/datasets/abedkhooli/arabic-bert-corpus) using [gensim](https://pypi.org/project/gensim/)

First, install the gensim package :

In [1]:
!pip install -q gensim

Import the necessary packages :

In [2]:
import gc
import spacy 
from spacy.lang.ar import Arabic
from spacy.tokenizer import Tokenizer
from pprint import pprint
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random
from gensim.test.utils import common_texts
from gensim.models import Word2Vec

Explore the corpus and print the path and name of some files :

In [3]:
# print one of each 50 lines
i = 0
files_list = []
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        files_list.append(os.path.join(dirname, filename))
        if (i % 50) == 0:
            print('file number ', i, ' : ', os.path.join(dirname, filename))
        i = i + 1

print('number of files in the corpus : ' , i)

file number  0  :  /kaggle/input/ar_bert32k-vocab.txt
file number  50  :  /kaggle/input/arwiki_books_shards/content/sharded/wiki_books_test_15.txt
file number  100  :  /kaggle/input/arwiki_books_shards/content/sharded/wiki_books_test_125.txt
file number  150  :  /kaggle/input/arwiki_books_shards/content/sharded/wiki_books_training_96.txt
file number  200  :  /kaggle/input/arwiki_books_shards/content/sharded/wiki_books_training_107.txt
file number  250  :  /kaggle/input/arwiki_books_shards/content/sharded/wiki_books_training_58.txt
file number  300  :  /kaggle/input/arwiki_books_shards/content/sharded/wiki_books_training_2.txt
file number  350  :  /kaggle/input/arwiki_books_shards/content/sharded/wiki_books_training_146.txt
file number  400  :  /kaggle/input/arwiki_books_shards/content/sharded/wiki_books_training_127.txt
file number  450  :  /kaggle/input/arwiki_books_shards/content/sharded/wiki_books_training_118.txt
file number  500  :  /kaggle/input/arwiki_books_shards/content/sharde

## Some explorations

In [4]:
# choose a random file to read
file_ = random.sample(files_list, 1)[0]

# print the path of the selected file
print('selected file :', file_)

# read this random file
data = pd.read_csv(file_, header=None, encoding='utf-8', lineterminator='\n')

# rename the 0 column
data = data.rename(columns={0: 'text'})

# shape of the dataframe
print('shape of dataframe : ', data.shape)

# take a look at the first rows
data.head()

selected file : /kaggle/input/arwiki_books_shards/content/sharded/wiki_books_training_81.txt
shape of dataframe :  (51914, 1)


,text
0,زيد بن محمد بن جعفر: المعروف بابن أبي اليابس ا...
1,حدث ببغداد عن: إبراهيم بن عبد الله القصار، ودا...
2,وعنه: ابن المظفر، وأبو حفص بن شاهين، وأبو الحس...
3,قال الخطيب: كان صدوقا.
4,حرف السين: سعيد بن إبراهيم بن معقل بن الحجاج: ...


As you can see, the text is reversed as Arabic reads from right to left unlike French or English for example.<br>
So let's try to create an other colum that will contain reversed text :

In [5]:
def reverse_text(x):
    x = list(reversed(x.split()))
    x = ' '.join(x)
    
    return x

data['text reversed'] = data['text'].apply(reverse_text)
data.head()

,text,text reversed
0,زيد بن محمد بن جعفر: المعروف بابن أبي اليابس ا...,الكوفي. الحسين أبو العامري، اليابس أبي بابن ال...
1,حدث ببغداد عن: إبراهيم بن عبد الله القصار، ودا...,الحمار. موسى بن وأحمد الحبري، الحكم بن والحسين...
2,وعنه: ابن المظفر، وأبو حفص بن شاهين، وأبو الحس...,رزقويه. بن الحسن وأبو شاهين، بن حفص وأبو المظف...
3,قال الخطيب: كان صدوقا.,صدوقا. كان الخطيب: قال
4,حرف السين: سعيد بن إبراهيم بن معقل بن الحجاج: ...,النسفي. عثمان أبو الحجاج: بن معقل بن إبراهيم ب...


Note that we have reversed the order of the words only. Normally we need to reverse the characters in each word, but we know that text processing is generally done on the word level only.

Add a column that counts the number of words :

In [6]:
data['words count'] = data['text reversed'].apply(lambda x : len(x.split()))
data.head()

,text,text reversed,words count
0,زيد بن محمد بن جعفر: المعروف بابن أبي اليابس ا...,الكوفي. الحسين أبو العامري، اليابس أبي بابن ال...,13
1,حدث ببغداد عن: إبراهيم بن عبد الله القصار، ودا...,الحمار. موسى بن وأحمد الحبري، الحكم بن والحسين...,20
2,وعنه: ابن المظفر، وأبو حفص بن شاهين، وأبو الحس...,رزقويه. بن الحسن وأبو شاهين، بن حفص وأبو المظف...,11
3,قال الخطيب: كان صدوقا.,صدوقا. كان الخطيب: قال,4
4,حرف السين: سعيد بن إبراهيم بن معقل بن الحجاج: ...,النسفي. عثمان أبو الحجاج: بن معقل بن إبراهيم ب...,12


Take a look at the smallest sentences :

In [7]:
data = data.sort_values(by='words count', ascending=True)
data.head()

,text,text reversed,words count
42571,قهار.,قهار.,1
3553,.,.,1
17672,ذكر.,ذكر.,1
21343,إناث.,إناث.,1
3598,.,.,1


Take a look at the longest sentences :

In [8]:
data = data.sort_values(by='words count', ascending=False)
data.head()

,text,text reversed,words count
51822,قوله عن عروة في رواية شعيب عن الزهري في العتق ...,/ لقوله والكافر المسلم بين ولاية ولا بالولاية ...,9488
51848,فاعلة يقال أبدت تأبد إذا توحشت ويقال جاء فلان ...,ولذكاء وللفهم للظهور يطلق البيان قوله بعض بيع ...,9006
51880,طفا طفا الشيء فوق الماء يطفو طفوا وطفوا ظهر وع...,/ العطاء وهو شكد جمع والأشكاد الوظيف عصبة أيضا...,8537
51782,والدارقطني والخطابي والحاكم في علوم الحديث وال...,قبله الذي للحديث مفسر وهو لسيده وينصح ربه عباد...,7540
51866,سحف سحف رأسه سحفا وجلطه وسلته وسحته حلقه فاستأ...,اسم وسيهف هم ذو الوجه ساهف أني الحزن من أصابني...,7104


Now, we will try to read the file into one text variable

In [9]:
# read the random file into a variable
with open(file_, 'r') as file:
    text = file.read()
    
# the first 1000 characters
text[0:1000]

'زيد بن محمد بن جعفر: المعروف بابن أبي اليابس العامري، أبو الحسين الكوفي.\nحدث ببغداد عن: إبراهيم بن عبد الله القصار، وداود بن يحيى الدهقان، والحسين بن الحكم الحبري، وأحمد بن موسى الحمار.\nوعنه: ابن المظفر، وأبو حفص بن شاهين، وأبو الحسن بن رزقويه.\nقال الخطيب: كان صدوقا.\nحرف السين: سعيد بن إبراهيم بن معقل بن الحجاج: أبو عثمان النسفي.\nرئيس أصحاب الحديث ببلده.\nسمع: أباه، ومعمر بن محمد البلخي.\nوبمكة: علي بن عبد العزيز، وغير واحد.\nوعنه: ابنه عبد الملك.\nوكان نقمة على القرامطة.\nحرف الشين: شعبة بن الفضل بن سعيد: أبو الحسن التغلبي.\nسمع: إدريس بن جعفر العطار، وبشر بن موسى، ومحمد بن عثمان بن أبي شيبة.\nوعنه: أبو الفتح بن مسرور لكن سماه سعيدا، وقال: لقبه شعبة؛ وعبد الرحمن بن عمر النحاس، وآخرون.\nوثقه الخطيب أبو بكر.\nوحدث بمصر، وبها مات.\nحرف العين: عبد الله بن عبد الرحمن بن أحمد بن حماد: الفقيه أبو العباس العسكري.\nسمع: محمد بن عبيد الله المنادي، وأبا داود السجستاني، وأحمد بن ملاعب، وجماعة.\nوعنه: ابن المظفر، والدارقطني، وابن رزقويه، وآخرون.\nتوفي في ربيع الأول.\nعبد الرحمن بن عمر بن سعي

In [10]:
# the last 1000 characters
text[-2000:]

'عد خلاف ومشادة كلامية مع الكابتن سعد عبد الحميد ورغم إصابته فانه لعب مباريات في المرحلة الأولى وسجل منها هدفين من مجموع أهدافه الأربعة في الدوري والتي كانت ضد أندية صلاح الدين وبيرس وهدفين ضد الحدود في المرحلتين الأولى والثانية وبعد انتهاء هذا الموسم جدد علاء بالتوقيع للنفط من جديد موسم وبقيادة المدرب حميد سلمان بعد أن أجرى عملية جراحية لساقه اليسرى وسجل في هذا الموسم في مرمى ميسان هدفين في المرحلة الأولى والثانية كما سجل ضد الحدود  كانت بدايته مع منتخب الناشئين بقيادة عدنان حمد الذي يعتبر أحد المدربين ممن هم أصحاب الفضل في اكتشاف موهبة علاء ستار إضافة إلى كريم جواد ورياض سالم ثم شارك مع كتيبة ثائر جسام في تصفيات أسيا للشباب في بنغلادش وسجل مع منتخبي الناشئين والشباب عدة أهداف أجملها على النيبال وحصل مع منتخب الشباب على كاس آسيا للشباب بقيادة عدنان حمد بعدها مع منتخبنا الرديف ببطولة الاستقلال التي أقيمت في جاكارتا وحصل على فضيتها بعد الخسارة من اندنوسيا في النهائي إضافة إلى الكثير من المحافل الخارجية الأخرى وخاض أول مبارياته الدولية رغم قلتها أمام قطر في الدوحة مع المدرب برند ستانج ال

We see that there is \n characters and . character in the text that can be used to separate sentences
Let's split the text by the . character first and see the results : 

In [11]:
text_dot = text.split('.')

# remove \n character from the sentences
text_dot = [x.replace('\n', '') for x in text_dot]

In [12]:
print('number of sentences using dot separator: ', len(text_dot))

number of sentences using dot separator:  68209


In [13]:
# take a look at the first lines
text_dot[0:5]

['زيد بن محمد بن جعفر: المعروف بابن أبي اليابس العامري، أبو الحسين الكوفي',
 'حدث ببغداد عن: إبراهيم بن عبد الله القصار، وداود بن يحيى الدهقان، والحسين بن الحكم الحبري، وأحمد بن موسى الحمار',
 'وعنه: ابن المظفر، وأبو حفص بن شاهين، وأبو الحسن بن رزقويه',
 'قال الخطيب: كان صدوقا',
 'حرف السين: سعيد بن إبراهيم بن معقل بن الحجاج: أبو عثمان النسفي']

In [14]:
# take a look at the last lines
text_dot[-5:]

['',
 ' وعلاء حاليا بعيد عن الملاعب نتيجة للإصابات المستمرة التي يعاني منها هذا النجم',
 '',
 ' والتي منعتنا من مشاهدة نجم كان يطلق عليه سابقا بأنه سيكون خليفة لأحمد راضي في الملاعب العراقية',
 '']

Note that the numbers (ex : dates) were removed from the corpus

In [15]:
# put this in a dataframe
data_2 = pd.DataFrame({'text' : text_dot, 'words count' : [len(x) for x in text_dot]})
data_2['text reversed'] = data_2['text'].apply(reverse_text)
data_2.sort_values(by='words count', ascending=False).head()

,text,words count,text reversed
67133,لا يذهب العرف بين الله والناس أو على أنه حال ...,212333,الهوى عن الدني النكس يوجد فاقتلوهفقد الرابعة أ...
67277,قال فكلمه بعض من حضر في قوله نضارة فقال أنشدن...,204037,أيكة حمامة بقادمتي تجلو ديوانه في وهو للنابغة ...
66026,فذكره سواء مختصرا الحديث الرابع قال النبي صلى...,116354,تعالى الله رحمه الناظم قال أعلم والله طير جوف ...
66599,وبنوا الصراط على زنة فعال لأنه يشتمل على سالك...,112951,عمره طول من يشكو القائل وهو الجاهلية في تعظمه ...
65651,هم الوشاة في الرضا والغضب عليهم اللعنة تترى و...,95094,الشاعر قال تبعكم أي وأشاعكم السلام شاعكم تقول ...


In [16]:
data_2.describe()

,words count
count,68209.000000
mean,167.647143
std,1596.144610
min,0.000000
25%,21.000000
50%,75.000000
75%,167.000000
max,212333.000000


Let's create a function that removes stop words and punctuation characters

In [17]:
def remove_sw_and_punc(text_):
    # free memory
#     gc.collect()
    nlp = Arabic()
    tokens  = nlp(text_)
    return [token for i, token in enumerate(tokens) if not token.is_stop and not token.is_punct]
    
    

In [18]:
# test the function
remove_sw_and_punc("ماهي أبرز التطورات السياسية، الأمنية والاجتماعية في العالم ؟")

[ماهي, أبرز, التطورات, السياسية, الأمنية, والاجتماعية, العالم]

# ****Training Arabic Corpus****
[credit](https://rare-technologies.com/word2vec-tutorial/)

In [19]:
# First clean memory
del data
del data_2
gc.collect()

0

In [20]:
class MySentences(object):
    def __init__(self, dirname, splitter = '\n'):
        self.dirname = dirname
        self.splitter = splitter
 
    def __iter__(self):
        for fname in os.listdir(self.dirname):
            # read file an split by \n
            with open(os.path.join(self.dirname, fname), 'r') as file:
                text = file.read()
            text = text.split(self.splitter)
            for line in text:
                # won't use it for now as we get memory exceeded error ! just split text
#                 line = remove_sw_and_punc(line) # remove stop words and punct. and returns a list of words
                line = line.split() # get the text in a list of words
                # reverse list because arabic reads from right to left !
                line = list(reversed(line))
                yield line

In [21]:
sentences = MySentences('/kaggle/input/arwiki_books_shards/content/sharded') # a memory-friendly iterator
model = Word2Vec(sentences)

In [22]:
model.save("word2vec.model")

In [23]:
# embedding of love
vector = model.wv['حب']
vector

array([ 1.0104702 ,  2.430672  ,  0.7731455 ,  0.77724564, -0.3688324 ,
       -0.02506618,  0.894535  ,  0.28758577, -1.6997569 , -1.4773976 ,
       -0.73162925,  1.2438555 ,  0.41841665, -1.0216689 , -0.37946466,
        3.5167632 , -0.23662491, -0.06168545,  1.4122976 , -0.62926173,
        4.3615584 ,  1.4523987 ,  2.0753465 , -0.27418175, -1.9324901 ,
       -0.45493454, -2.8136609 ,  0.10908411, -0.9994772 , -1.4642388 ,
        1.2633066 ,  0.6484049 , -0.8990293 , -1.5088811 , -0.0838986 ,
        0.9348607 ,  1.2784283 , -0.11465404,  0.8257239 , -2.1521156 ,
        0.99173135, -1.6406078 , -0.79281455,  0.6000387 , -4.028433  ,
       -0.8285572 , -2.851047  ,  1.2855773 ,  2.1874597 ,  1.746882  ,
       -0.37815326, -1.1073989 , -0.5202484 ,  5.4930716 , -0.24959347,
        1.6950998 , -1.8424301 , -2.096298  , -3.5287833 ,  0.25051892,
       -0.40514553, -3.7662165 ,  4.0990977 , -0.9424642 , -0.26397339,
       -0.8065114 ,  2.4079704 , -1.277007  ,  1.9470444 , -1.05

In [24]:
sims = model.wv.most_similar('حب', topn=10)
print(sims)

[('وحب', 0.785637378692627), ('فحب', 0.7165203094482422), ('الحب', 0.6648672819137573), ('حبه', 0.6577664017677307), ('محبة', 0.6569615602493286), ('والحب', 0.6508806943893433), ('حبا', 0.6284101605415344), ('لحب', 0.6230635046958923), ('وعشق', 0.6184947490692139), ('بحب', 0.6156246662139893)]


In [25]:
# embedding of evolutions ('التطورات')
vector = model.wv['التطورات']
vector

array([ 0.73679954, -5.364433  ,  2.3980696 , -0.5434981 ,  2.8935876 ,
        2.493084  ,  0.0507003 , -1.9021779 ,  0.8683302 ,  0.5848708 ,
        1.6783851 ,  0.26133633,  0.06136704, -0.5810713 , -0.17748466,
        0.29832548,  0.5005483 ,  0.6732952 ,  0.8383038 ,  3.41848   ,
        1.0394856 ,  0.16484839, -0.3858048 , -0.659515  ,  6.591456  ,
        3.6856885 ,  0.34697172, -1.0669267 , -0.6228201 ,  1.3555706 ,
        0.5860797 , -0.9287194 ,  2.5301907 ,  1.0510671 , -3.3038003 ,
       -3.12347   , -1.8136512 ,  2.12472   , -1.8231436 ,  0.7365643 ,
        0.4483917 ,  0.40165588,  1.4579067 ,  2.949197  ,  2.0059729 ,
        2.8807151 , -0.9716648 , -4.0685678 , -0.5472251 , -0.3914817 ,
        1.4372985 ,  0.23140708, -1.3177879 , -2.0943913 ,  0.14405084,
       -2.558188  , -2.7722208 ,  1.0180699 , -2.5434153 ,  1.7598586 ,
       -1.2936295 ,  1.1624461 ,  1.382615  ,  0.5319775 , -0.07660569,
       -0.03359473, -1.1232122 ,  7.3378267 ,  0.98226464,  0.65

In [26]:
sims = model.wv.most_similar('التطورات', topn=10)
print(sims)

[('التحولات', 0.8750647306442261), ('التغييرات', 0.8372082710266113), ('الابتكارات', 0.8325063586235046), ('التغيرات', 0.8191807270050049), ('الاكتشافات', 0.8177353143692017), ('التحليلات', 0.8168216347694397), ('الإنجازات', 0.8126815557479858), ('النقاشات', 0.8104127645492554), ('المستجدات', 0.8078758716583252), ('الاختراعات', 0.8067344427108765)]
